<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [73]:
# Doing some imports
import gensim
import os
import zipfile
import pandas as pd
from gensim.parsing import preprocessing 
from gensim.models.phrases import Phrases
from gensim.corpora.dictionary import Dictionary
import spacy
from gensim.models import LdaMulticore

In [6]:
# path to the zip file
ZIP_PATH = os.path.join("..", "module1-text-data", "data", "Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip")

In [7]:
# Will be pulling in the data
with zipfile.ZipFile(ZIP_PATH, 'r') as myzip:
    with myzip.open("Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv") as myFile:
        df = pd.read_csv(myFile)

In [8]:
df.head(3)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."


In [9]:
reviewList  = df["reviews.text"].values.tolist()

In [10]:
# Looking at the list
reviewList[:10]

['I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.',
 'Bulk is always the less expensive way to go for products like these',
 'Well they are not Duracell but for the price i am happy.',
 'Seem to work as well as name brand batteries at a much better price',
 'These batteries are very long lasting the price is great.',
 "Bought a lot of batteries for Christmas and the AmazonBasics Cell have been good. I haven't noticed a difference between the brand name batteries and the Amazon Basic brand. Just a lot easier to purchase and have arrive at the house and have on hand. Will buy again.",
 'ive not had any problame with these batteries have ordered them in the past been very pleased.',
 'Well if you are looking for cheap non-rechargeable batteries that last quite a while then these are perfect. Nothing more to say.',
 'These do not hold the amount of high power juice like energizer or duracell, but th

In [11]:
practice_string = ["I love to eat cat's and their funny", "What? how come you didn't tell me?"]

In [12]:
#Trying to use the pre_process string
def myProcess_function(doc):
    processed_list = []
    # Getting a tokenizer from the spacy


    for sentance in doc:
        sentance = preprocessing.remove_stopwords(sentance)
        sentance = preprocessing.strip_numeric(sentance)
        sentance = preprocessing.strip_punctuation(sentance)
        sentance = preprocessing.strip_short(sentance)
        #tokens = preprocess_string(sentance)
        processed_list.append(sentance)
    return processed_list

In [13]:
myProcess_function(practice_string)

['love eat cat funny', 'What come didn tell']

In [22]:
# Trying it another way
def process_docs(doc):
    processed_text = []
    FILTER = [lambda x: x.lower(), preprocessing.remove_stopwords, preprocessing.strip_punctuation, preprocessing.strip_multiple_whitespaces, preprocessing.strip_numeric, preprocessing.strip_short]
    for text in doc:
        text = preprocessing.preprocess_string(text, FILTER)
        processed_text.append(text)
    return processed_text

In [66]:
# This is showing doing a practice on two strings
process_docs(practice_string)

[['love', 'eat', 'cat', 'funny'], ['what', 'come', 'didn', 'tell']]

In [27]:
# Trying to add bigrams to the list of # tokens
# This is the method that will make the bigram phrases
def make_bigram(text):
    bigram_data = Phrases(text, min_count=5, threshold=10.0)
    data = [bigram_data[doc] for doc in text]
    return data

In [16]:
# loading the lemmatizer from spacy
nlp = spacy.load("en_core_web_lg", disable=['parser', 'ner'])

In [116]:
# Adding the lemmatizer of spacy here
def do_lemmetization(body_of_texts_list, allowed_pos = ['NOUN', 'ADJ', 'VERB', 'ADV']):
    for text in body_of_texts_list:
        # joining all the contents of the list back to one string
        text = " ".join(text)
        doc = nlp(text)
        j = [token.lemma_ for token in doc if token.pos_ in allowed_pos]
        yield j

In [117]:
# Calling all of these in a row 
# Will try fist with my practic string
processed_doc = process_docs(practice_string)
bigram_data = make_bigram(processed_doc)
print(bigram_data)
my_generator = do_lemmetization(bigram_data)
my_generator

[['love', 'eat', 'cat', 'funny'], ['what', 'come', 'didn', 'tell']]


<generator object do_lemmetization at 0x00000255867CF930>

In [118]:
# Creating the dictcionary from gensim
id_word_dictionary = Dictionary(my_generator)
id_word_dictionary.filter_extremes(no_below=0, no_above=.95)
print(id_word_dictionary.token2id["love"])

3


In [119]:
# Now doing the real thing on the real data
processed_doc = process_docs(reviewList)
bigram_data = make_bigram(processed_doc)

my_generator = do_lemmetization(bigram_data)
my_generator

<generator object do_lemmetization at 0x00000255867CF750>

In [120]:
# Creating the dictionary and the bag_of_words(count Vect)
id_word_dictionary = Dictionary(my_generator)
# Doing some statistical trimming
id_word_dictionary.filter_extremes(no_below=2, no_above=.95)


In [106]:
the_generator = do_lemmetization(bigram_data)


In [80]:
next(the_generator)

['order',
 'item',
 'bad',
 'quality',
 'miss',
 'backup',
 'spring',
 'pc',
 'aluminum',
 'battery',
 'work']

In [115]:
for j in the_generator:
    len(j)

In [121]:
# Building the bag of words, if the length of the doc is 0 after doing trimming and removing 
# stop words then that review will not be used
bag_corpus = [id_word_dictionary.doc2bow(doc) for doc in do_lemmetization(bigram_data) if len(doc) != 0]


In [122]:
len(bag_corpus)

28000

In [123]:
bag_corpus[345]

[(10, 1), (21, 1)]

In [130]:
# Building the model 
lda = LdaMulticore(corpus=bag_corpus, id2word=id_word_dictionary, 
workers=3, num_topics=8, passes=10 )

In [131]:
lda.print_topics()

[(0,
  '0.043*"use" + 0.028*"download" + 0.026*"buy" + 0.025*"tablet" + 0.020*"time" + 0.020*"app" + 0.018*"love" + 0.012*"able" + 0.012*"work" + 0.011*"great"'),
 (1,
  '0.022*"use" + 0.022*"love" + 0.021*"light" + 0.020*"kindle" + 0.016*"good" + 0.015*"easy" + 0.014*"reader" + 0.013*"new" + 0.013*"read" + 0.011*"reading"'),
 (2,
  '0.138*"great" + 0.106*"tablet" + 0.058*"price" + 0.052*"good" + 0.034*"product" + 0.023*"work" + 0.021*"kid" + 0.020*"love" + 0.018*"nice" + 0.012*"need"'),
 (3,
  '0.020*"buy" + 0.016*"easy_use" + 0.012*"problem" + 0.011*"purchase" + 0.011*"replace" + 0.011*"granddaughter" + 0.010*"use" + 0.010*"year" + 0.009*"charge" + 0.009*"day"'),
 (4,
  '0.103*"love" + 0.078*"buy" + 0.052*"kid" + 0.027*"great" + 0.026*"get" + 0.026*"tablet" + 0.022*"purchase" + 0.019*"daughter" + 0.018*"year_old" + 0.016*"gift"'),
 (5,
  '0.050*"tablet" + 0.035*"read" + 0.033*"book" + 0.030*"screen" + 0.024*"easy" + 0.017*"use" + 0.015*"app" + 0.013*"well" + 0.013*"want" + 0.012*"siz

In [126]:
# Getting the visualizations
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [132]:
pyLDAvis.gensim.prepare(lda, bag_corpus, id_word_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.041658  0.278775       1        1  24.011833
2      0.236371  0.027451       2        1  13.924000
1     -0.041722 -0.050748       3        1  11.738661
5      0.008256 -0.138297       4        1  11.504004
4      0.143170 -0.089132       5        1  11.113719
3     -0.193108  0.084132       6        1  11.058097
0     -0.024346 -0.062849       7        1   8.505204
7     -0.170280 -0.049331       8        1   8.144480, topic_info=          Term         Freq        Total Category  logprob  loglift
3      battery  8397.000000  8397.000000  Default  30.0000  30.0000
19       great  8430.000000  8430.000000  Default  29.0000  29.0000
2788    tablet  6413.000000  6413.000000  Default  28.0000  28.0000
74        love  4799.000000  4799.000000  Default  27.0000  27.0000
17       price  4690.000000  4690.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
832   year_old   127.286102   629.227966   Topic8  -5.0773   0.9098
168       hour   111.954170   388.683258   Topic8  -5.2056   1.2632
29    purchase   142.850113  2220.284424   Topic8  -4.9619  -0.2358
151        son   113.224495   600.376343   Topic8  -5.1943   0.8396
183        new   112.991158   922.091553   Topic8  -5.1964   0.4085

[626 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
93        1  0.978945          aaa
93        6  0.017025          aaa
1039      1  0.005640         able
1039      2  0.065803         able
1039      3  0.062043         able
...     ...       ...          ...
921       2  0.234269        young
921       4  0.052060        young
921       5  0.696301        young
921       6  0.006507        young
4189      2  0.919239  young_child

[1406 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 3, 2, 6, 5, 4, 1, 8])

I Tried a few different amounts of topics and settled with 8.  I tried to see what I could lable each topic with and am still not sure.  I did find that some of the topics dealt more with the specs of kindles and batteries while another topic grouped together the emotions that people have when choosing products.

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling